In [1]:
import pandas as pd
import random
#Read 1% of dataset
readProportion = 0.01
data = pd.read_table('https://datasets.imdbws.com/title.ratings.tsv.gz',
              header=0, skiprows=lambda x: x > 0 and random.random() > readProportion,
              sep='\t')

In [2]:
#Get names for each titleid
names = pd.read_table('https://datasets.imdbws.com/name.basics.tsv.gz', sep='\t')

In [3]:
#Filter to only actors and actresses
namesAct = names[names.primaryProfession.str.contains('actor|actress', na=False)]

In [4]:
#Make titles known for into list
titlesKnown = namesAct.knownForTitles.copy().str.split(pat=',')

In [30]:
#Creates actor and titles dataframe
nameTitles = pd.concat([titlesKnown, namesAct.nconst],
          ignore_index=True, axis=1)
nameTitles.rename(columns={0:'tconst', 1:'nameID'}, inplace=True)

In [6]:
#Read title basics dataset
titles = pd.read_table("https://datasets.imdbws.com/title.basics.tsv.gz", sep="\t")

/Users/nsortur/.conda/envs/untitled/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3063: DtypeWarning: Columns (5) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [7]:
movies = titles[titles.titleType == 'movie'].copy()

In [8]:
deleteCols = ['originalTitle',
              'isAdult',
              'startYear',
              'endYear',
              'runtimeMinutes',
              'genres',
              'titleType']
movies.drop(deleteCols, axis=1, inplace=True)

In [35]:
#Append nameIDs onto each tconst in "movies"
firstTitle = nameTitles['tconst'].apply(lambda x: x[0])
firstTitleName = pd.concat([firstTitle, nameTitles['nameID']], axis=1)

In [36]:
nameMovie = movies.merge(firstTitleName, on='tconst')

In [41]:
test = nameMovie.groupby('tconst').nameID.apply(list).reset_index()